In [1]:
import pandas as pd
import requests
import json
from datetime import datetime, date
import time

In [12]:
cities = [    # 10 German cities by population
         "Q64",    # Berlin: federal state, capital and largest city of Germany
         "Q1055",  # Hamburg: city and federal state in the North of Germany
         "Q1726",  # München: capital and most populous city of Bavaria, Germany
         "Q365",   # Köln: city in North Rhine-Westphalia, Germany
         "Q1794",  # Frankfurt am Main: city in Hesse, Germany
         "Q1022",  # Stuttgart: capital city of German federated state Baden-Württemberg
         "Q1718",  # Düsseldorf: capital city of the German federated state of North Rhine-Westphalia
         "Q1295",  # Dortmund: city in North Rhine-Westphalia, Germany
         "Q2066",  # Essen: city in North Rhine-Westphalia, Germany
         "Q2079",  # Leipzig: most populous city in the German state of Saxony
                  
              # 5 Austrian cities by population
         "Q1741",  # Wien: capital of and state in Austria
         "Q13298", # Graz: capital of Styria, Austria
         "Q41329", # Linz: capital city of Upper Austria, Austria
         "Q34713", # Salzburg: capital city of the federal state of Salzburg in Austria
         "Q1735"   # Innsbruck: capital of the state of Tyrol, Austria
         ]

In [17]:
import requests

url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q64"

headers = {
	"X-RapidAPI-Key": "API key goes here#",
	"X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

#print(response.text)

In [15]:
# loop through the list of cities, makes API calls and concatenate all city data into one dataframe

city_list =[]
for c in cities:
    url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{c}"
    headers = {
        "X-RapidAPI-Key": "API key goes here#",
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers=headers)
    #time.sleep(2)
    city_df = pd.json_normalize(response.json())
    city_list.append(city_df)
    
cities_df = pd.concat(city_list, ignore_index=True)
cities_df = cities_df[["data.id", "data.wikiDataId", "data.city", "data.country", "data.elevationMeters", 
                       "data.latitude", "data.longitude", "data.population"]]

#cities_df

In [ ]:
# rewrite the above as a function so it can be implemented elsewhere

def get_city_data(cities):
    city_list = []
    url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q60"
    headers = {
        "X-RapidAPI-Key": " #API key goes here# ",
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
    }
    test = requests.request("GET", url, headers = headers)
    time.sleep(1)
    if test.status_code >= 200 and test.status_code <= 299:
        for city in cities:
            url = f"https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{city}"
            headers = {
                "X-RapidAPI-Key": "#API key goes here# ",
                "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
            }
            response = requests.request("GET", url, headers = headers)
            time.sleep(2)
            city_df = pd.json_normalize(response.json())
            city_list.append(city_df)
    else:
        return -1
    cities_df = pd.concat(city_list, ignore_index = True)
    return cities_df

In [ ]:
# this returns the full set of columns from the API call

city_data = get_city_data(cities)
#city_data

In [36]:
# reset the dataframe to exclude unnecessary columns

city_data = city_data[["data.wikiDataId", "data.city", "data.country", "data.countryCode", "data.region", 
                       "data.elevationMeters", "data.latitude", "data.longitude", "data.population"]]

In [20]:
# rename columns with more intuitive names and get rid of dots to avoid problems in MySQL

city_data.rename(columns = {"data.wikiDataId": "city_id",
                            "data.city": "city" ,
                            "data.country": "country",
                            "data.countryCode": "country_code",
                            "data.region": "region",
                            "data.elevationMeters": "elevation",
                            "data.latitude": "city_latitude",
                            "data.longitude": "city_longitude",
                            "data.population": "population"},
                 inplace = True)
city_data

In [39]:
city_data.dtypes

city_id            object
city               object
country            object
country_code       object
region             object
elevation          object
city_latitude     float64
city_longitude    float64
population          int64
dtype: object

In [41]:
city_data.to_csv("city_data.csv")